In [7]:
import numpy as np
import networkx as nx

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap



In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
#Choose a title!
title = 'MST'


msts = np.load('data/msts.npy', allow_pickle='TRUE').item()
noisy_msts = np.load('data/noisy_msts.npy', allow_pickle='TRUE').item()


mst = msts[list(sorted(msts.keys()))[0]]


#node_color = [nx.degree(mst)[n] for n in nx.nodes(mst)]
#node_size = [400 + nx.degree(mst)[n] * 20 for n in nx.nodes(mst)]
#pos = nx.kamada_kawai_layout(mst)
#pos = nx.spring_layout(mst)
#nx.draw(mst, pos, node_color=node_color, cmap='Oranges', labels=nx.get_node_attributes(mst, 'ticker'), with_labels=True, node_size=node_size, font_size=8)


degrees = dict(nx.degree(mst))
nx.set_node_attributes(mst, name='degree', values=degrees)

base_size = 10
adjusted_node_size = dict([(node, degree+base_size) for node, degree in nx.degree(mst)])
nx.set_node_attributes(mst, name='adjusted_node_size', values=adjusted_node_size)

size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

color_palette = Blues8

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = HOVER_TOOLTIPS = [
      ("Ticker", "@ticker"),
        ("Degree", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)


#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(mst, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)